In [2]:
import pandas as pd
import reverse_geocoder as rg
from geopy.geocoders import Nominatim 
from googletrans import Translator, constants
translator = Translator()
geolocator = Nominatim(user_agent="geoapiExercises")

df = pd.read_csv('bibliography.csv', delimiter=',', encoding="utf-8")


In [6]:
# e_data = pd.read_csv("https://www.dropbox.com/s/u2xupuhf07y85zh/earthQuake.csv?dl=1")
# density_data = pd.read_csv("https://www.dropbox.com/s/p6ejr4mzx8pjq9m/population_density_long.csv?dl=1")
# population_data=pd.read_csv("https://www.dropbox.com/s/bp5hb61zrui2qo6/population_total_long.csv?dl=1")

In [11]:
e_data_with_country = pd.read_csv('out_e_data.csv', delimiter=',', encoding="utf-8")
density_data = pd.read_csv('population_density_long.csv', delimiter=',', encoding="utf-8")
population_data=pd.read_csv('population_total_long.csv', delimiter=',', encoding="utf-8")

separate the date into columns

In [10]:
# e_data[['Month','Day','Year']]=e_data['Date'].str.split('/',-1,expand=True)

NameError: name 'e_data' is not defined

get rid of all unwanted columns (is Depth usefull?)

In [ ]:
# e_data = e_data.drop(['Date','Time','ID','Source','Location Source','Magnitude Source','Status','Type','Magnitude Type'],axis=1)

In [ ]:
# print(e_data)

In [12]:
population_data = population_data.drop(population_data.index[population_data['Year'].isin(['1960', '1961', '1962','1963','1964'])])
population_data.rename(columns={'Count': 'Population'}, inplace=True)

In [13]:
density_data = density_data.drop(density_data.index[density_data['Year'].isin(['1960', '1961', '1962','1963','1964'])])
density_data.rename(columns={'Count': 'Density'}, inplace=True)

In [ ]:
def find_country(long,lat):
    tuple_input = (long,lat)
    found = rg.search(tuple_input)
    countryName = found[0]['name']
#     countryName = translator.translate(countryName)
    print('**', countryName)
    
    return countryName

def city_to_country(city):
    try:
        loc = geolocator.geocode(city, language='en')
        loc_list = loc.address.split(',')
    except:
        print(city + ': Not found')
        return None
    return loc_list[-1]
    

In [14]:
joined = density_data.merge(population_data,
          how='left',
          on =['Country Name', 'Year'],
         )
joined['Population'] = joined['Population'].fillna(0).astype(int)
print(joined)
pop_dense_data = joined
#  suffixes=['', '_r']).drop(columns = ['date_r', 'year-month']

       Country Name  Year  Density  Population
0             Aruba  1965      318       57360
1       Afghanistan  1965       15     9956320
2            Angola  1965        4     5770570
3           Albania  1965       68     1864791
4           Andorra  1965       39       18549
...             ...   ...      ...         ...
11230        Kosovo  2017      168     1830700
11231   Yemen, Rep.  2017       52    27834821
11232  South Africa  2017       46    57000451
11233        Zambia  2017       22    16853688
11234      Zimbabwe  2017       36    14236745

[11235 rows x 4 columns]


In [ ]:
# count = 0
# for i, r, in e_data.iterrows():
#     country = city_to_country(find_country(r['Latitude'], r['Longitude']))
#     e_data['Country Name'][count] = country
#     print(country)
#     count +=1
    

In [18]:
e_data_with_country = e_data_with_country.dropna()
print(e_data_with_country)

       Unnamed: 0  Latitude  Longitude   Depth  Magnitude Month   Day    Year  \
0               0   19.2460   145.6160  131.60        6.0    01   2.0  1965.0   
2               2  -20.5790  -173.9720   20.00        6.2    01   5.0  1965.0   
3               3  -59.0760   -23.5570   15.00        5.8    01   8.0  1965.0   
4               4   11.9380   126.4270   15.00        5.8    01   9.0  1965.0   
5               5  -13.4050   166.6290   35.00        6.7    01  10.0  1965.0   
...           ...       ...        ...     ...        ...   ...   ...     ...   
23407       23407   38.3917  -118.8941   12.30        5.6    12  28.0  2016.0   
23408       23408   38.3777  -118.8957    8.80        5.5    12  28.0  2016.0   
23409       23409   36.9179   140.4262   10.00        5.9    12  28.0  2016.0   
23410       23410   -9.0283   118.6639   79.00        6.3    12  29.0  2016.0   
23411       23411   37.3973   141.4103   11.94        5.5    12  30.0  2016.0   

                           

In [26]:

fixed_e_data = e_data_with_country.astype({"Year": int, "Country Name": str})
fixed_pop_data = pop_dense_data.astype({"Year": int, "Country Name": str})
# fixed_e_data['Country Name'] = fixed_e_data['Country Name'].astype("|S")


In [30]:
firstRun = fixed_e_data.loc[(fixed_e_data['Country Name'] == "United States")].count()
print(fixed_e_data.dtypes)
print(firstRun)

Unnamed: 0        int64
Latitude        float64
Longitude       float64
Depth           float64
Magnitude       float64
Month            object
Day             float64
Year              int32
Country Name     object
dtype: object
Unnamed: 0      0
Latitude        0
Longitude       0
Depth           0
Magnitude       0
Month           0
Day             0
Year            0
Country Name    0
dtype: int64


In [ ]:
# df['column'] = df['column'].astype('|S80')

In [28]:
joined = e_data_with_country.merge(fixed_pop_data,
           how='inner',
          on =['Country Name', 'Year'],
         )
joined['Population'] = joined['Population'].fillna(0).astype(int)
# joined['Density'] = joined['Density'].fillna(0).astype(int)
print(joined)


    Unnamed: 0  Latitude  Longitude  Depth  Magnitude Month   Day    Year  \
0          587  -48.8730   112.8030   15.0        6.0    01   7.0  1967.0   
1         1503   27.7710    54.5400   25.0        5.8    02  23.0  1970.0   
2         1902   -4.3300   102.2850   75.0        7.0    04   8.0  1971.0   
3         2381   -4.1190   102.2800   90.0        5.6    08   1.0  1972.0   
4         2483   63.1260  -150.9680  130.4        5.5    10  21.0  1972.0   
..         ...       ...        ...    ...        ...   ...   ...     ...   
68       20764   -4.8930   101.8480   26.0        5.6    05   4.0  2011.0   
69       21316  -15.8770   -72.4130  110.0        6.1    06   7.0  2012.0   
70       21409  -50.1430   114.0800   10.0        5.6    08  30.0  2012.0   
71       23314    3.4466   -74.4951   33.0        5.6    10  31.0  2016.0   
72       23321  -50.0472   114.3817   10.0        5.7    11  10.0  2016.0   

    Country Name  Density  Population  
0        Denmark      114     48353

In [ ]:
# gg = pop_dense_data.join(ee_data, on =['Country Name', 'Year'], how='left', lsuffix='', rsuffix='', sort=False)